In [26]:
def GT_Patchify(slidepath, annotpath, des, m_level=0):

    import sys, numpy as np, matplotlib.pyplot as plt,cv2 as cv,os,openslide,xml.etree.cElementTree as ET,random,glob

    sys.path.append("C:/Program Files/ASAP 2.0/bin")

    import multiresolutionimageinterface as mir

    factor = 0**m_level
    slidename = os.listdir(slidepath)
    annotname = os.listdir(annotpath)
    
    def parse_xml(anno_path):
        tree = ET.ElementTree(file=anno_path)
        annolist={}
        root = tree.getroot()
        i = 0
        for coords in root.iter('Coordinates'):
            vasc = []
            for coord in coords:
                vasc.append((int(float(coord.attrib.get("X"))/factor),int(float(coord.attrib.get("Y"))/factor)))
            annolist[i] = vasc
            i+=1
        return annolist
    
    def generate_patches(slidepath,slidename,annotpath, annotname,des, patch_size):
        slide_path = os.path.join(slidepath, slidename)
        anno_path = os.path.join(annotpath,annotname)
        Slide =  openslide.OpenSlide(slide_path)
        annolist = parse_xml(anno_path)
        for i in range(len(annolist)):                   
            coords = np.array(annolist[i])
            x, y, w, h = cv.boundingRect(coords)
            for k in range(30):                        
                offsetx = random.randrange(-1000, 100)
                offsety = random.randrange(-1000, 100)
                spointx, spointy = x + offsetx, y + offsety 
                spointx0, spointy0 = spointx*factor, spointy*factor 
                patching = Slide.read_region((spointx0, spointy0), m_level, (patch_size, patch_size))                        
                cv.imwrite(des + f"{slidename.split('.')[0]}_lvl{m_level}_patch{str(i)}{str(k)}.jpg", np.array(patching))
    des = des + 'GroundTruths/'
    os.makedirs(des)
    for i in range(len(slidename)):
        annolist = parse_xml(os.path.join(annotpath,annotname[i]))
        generate_patches(slidepath,slidename[i],annotpath,annotname[i],des,512)

In [27]:
slidepath = 'D:/Practice/Lung_cancer/WSIs/'  #path to folder of WSIs
annotpath = 'D:/Practice/Lung_cancer/Anns/'
des = 'D:/Practice/Lung_cancer/'            #Ground truth folder will itself be created in pasted directory
GT_Patchify(slidepath, annotpath, des)

KeyboardInterrupt: 